<a href="https://colab.research.google.com/github/cimbelli/Inland_Navigable_Waterway/blob/main/s1_danube_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Number of vessels in Serbian navigable waterways 

In [40]:
### comment below after the first run

#!pip install geojson
#!pip install earthengine-api

url = 'https://raw.githubusercontent.com/cimbelli/Inland_Navigable_Waterway/main/aoi2.geojson'
!wget $url


--2022-09-28 22:55:09--  https://raw.githubusercontent.com/cimbelli/Inland_Navigable_Waterway/main/aoi2.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1694521 (1.6M) [text/plain]
Saving to: ‘aoi2.geojson’

aoi2.geojson        100%[===================>]   1.62M  --.-KB/s    in 0.04s   

2022-09-28 22:55:09 (38.9 MB/s) - ‘aoi2.geojson’ saved [1694521/1694521]



In [3]:
import ee
#import time
import matplotlib
import numpy as np
import requests
#import geemap
#from urllib.request import urlopen
#import urllib

import os
import time
import geojson
import datetime, calendar
import json
import csv

import folium
#import geopandas as gpd

In [4]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
           
def download(url, tipo, name):
    response = requests.get(url)
    
    if tipo == "raster":
        imgref = os.path.join(name + ".tif")
        with open(imgref, 'wb') as fd:
            fd.write(response.content)
    else:
        vec = os.path.join(name + ".geojson")
        with open(vec, 'wb') as fd:
            fd.write(response.content)
            
    return None

def download_raster(img, name):
    url = img.getDownloadUrl({
        'bands': ['VV'],
        'region': bb,
        'scale' : 10,
        'format': 'GEO_TIFF',
        'maxPixels': 834607840,
        'formatOptions': {
            'cloudOptimized': True
          }
    })
    download(url,"raster", name)
    return None

In [5]:
#start_time = time.time()
# Trigger the authentication flow.

ee.Authenticate()



# Initialize the library.
ee.Initialize()
maxPixels: 6362493000



image_viz_params = {
    'bands': ['VV'],
    'min': 0.0,
    'max': 1.0,
    #'gamma': [0.95, 1.1, 1]
}

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=0pe8O5MD1-QxuqPtrOPoPBZLSYMOwsUP71WY_XnbdHs&tc=aQfto1UWOFcsatLoIJ_dcMQEb7oyz9H3I8Jvc34ynQo&cc=IbggZpNGT9Ia5fsfQgKZ2G1NrZ7Vw-QMD1YNWCdCZpg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJpIg1v94RqAy-z-tto6gzMSo-pqmkUPP6--IZwTyqoCoCd3RAwv3Co

Successfully saved authorization token.


In [42]:
f = open('aoi2.geojson')
data1 = json.load(f)

csvfilename = "Serbia_vessels.csv"
header = ['year', 'month', 'vessels', 'imgs']
with open(csvfilename, 'w', encoding='UTF8') as f1:
    writer = csv.writer(f1)
    writer.writerow(header)

    for year in range (2022, 2023):

        for m in range(1,13):
            
            tot_vessels = 0
            tot_images = 0
            for i in range(len(data1['features'])):
                print(i)
                coords = data1['features'][i]['geometry']['coordinates']
                try:
                    [j.pop(2) for j in coords[0][0]]
                except:
                    print("except")
                
                aoi = ee.Geometry.Polygon(coords[0][0])

                m1 = ('0' if m <10 else'') + str(m)
                date1 = str(year) + '-' + m1 + '-01'
                date2 = str(year) + '-' + m1 + '-' + str(calendar.monthrange(int(year), m)[1])
                
                #if id1 > 20:
                    #print(id1)

                imgVV = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')   #S1_GRD_FLOAT
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) 
                        .filter(ee.Filter.eq('platform_number', 'A'))
                        .filter(ee.Filter.eq('instrumentMode', 'IW'))
                        #.filter(ee.Filter.Or(ee.Filter.eq('relativeOrbitNumber_start', '153'),ee.Filter.eq('relativeOrbitNumber_start', '175')))
                        
                        .select('VV')
                        .filterDate(date1, date2)
                        .filterBounds(aoi)
                        #.reduce(ee.Reducer.percentile([50]))
                        )
                imgVV1 = imgVV.filterMetadata('relativeOrbitNumber_start', 'equals', 175)
                imgVV1 = imgVV1.filterMetadata('relativeOrbitNumber_end', 'equals', 175)
                imgVV1 = imgVV1.filterMetadata('sliceProductFlag', 'equals', True)
                imgVV1 = imgVV1.filterMetadata('sliceNumber', 'equals', 20)

                #imgVV1 = imgVV

                imgVV_fixed = imgVV1.max().clip(aoi).toUint8()
                im = imgVV_fixed

                center = list(aoi.centroid().getInfo().values())[1][1], list(aoi.centroid().getInfo().values())[1][0]

                classes = im.reduceToVectors(
                    **{
                    'reducer': ee.Reducer.countEvery(),
                    'geometry': aoi.getInfo(),
                    'scale': 10,
                    'maxPixels': 100000000
                })

                vessels = (ee.FeatureCollection(classes).filter(ee.Filter.gt('count', 15)).filter(ee.Filter.lt('count', 10000)))
                vessels_centroids = vessels.map(lambda f: f.centroid(10))

                #vessels_centroids1 = vessels_centroids.intersection(aoi)
                vessels_centroids1 = vessels_centroids.filter(ee.Filter.bounds(aoi))
                
                
                n_vessels = vessels_centroids1.size().getInfo()
                n_imgs = imgVV1.size().getInfo()
                #dens = round(100*n_vessels/n_imgs,2)
                
                
                tot_vessels += tot_vessels + n_vessels
                tot_images += tot_images + n_imgs

            #print("there are", n_vessels, "vessels", "n_img", imgVV.size().getInfo())
            writer.writerow([year, m, tot_vessels, tot_images])
            
            
            #downloadUrl1 = vessels_centroids1.getDownloadURL(**{'filetype': 'GEOJSON'})
            #download(downloadUrl1, "vessels", "vessels_" + str(m) + "_" + str(year))
            print(year, m, tot_vessels, tot_images)

print("finito")



0
except


EEException: ignored

In [44]:
aoi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              19.71708,
              44.752096
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})